# 下载模型与数据集

识别模型：ds4sd/docling-models

In [ ]:
import os
# 设置镜像地址
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from huggingface_hub import snapshot_download
from huggingface_hub.utils import HfHubHTTPError

def download_docling_models(target_dir, repo_id="ds4sd/docling-models"):
    """
    将指定的 HuggingFace 仓库下载到本地目录。
    
    参数:
        target_dir (str): 本地保存模型的路径。
        repo_id (str): HuggingFace 上的仓库 ID。
    """
    print(f"准备开始下载 '{repo_id}' ...")
    print(f"目标保存路径: {os.path.abspath(target_dir)}")

    try:
        # snapshot_download 会下载整个仓库
        # local_dir_use_symlinks=False 确保下载的是实际文件而不是缓存的软链接
        local_path = snapshot_download(
            repo_id=repo_id,
            local_dir=target_dir,
            local_dir_use_symlinks=False, 
            resume_download=True,  # 支持断点续传
            # max_workers=4        # 可选：如果网速允许，增加并发数
        )
        print(f"✅ 下载成功！文件已保存在: {local_path}")
        
    except HfHubHTTPError as e:
        print(f"❌ 下载失败 (网络或权限错误): {e}")
    except Exception as e:
        print(f"❌ 发生未知错误: {e}")

if __name__ == "__main__":
    # --- 配置区域 ---
    
    # 你想保存模型的本地路径
    LOCAL_DIRECTORY = "./model/docling" 
    
    # 运行下载函数
    download_docling_models(LOCAL_DIRECTORY)

d:\program\project\python\MHier-RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


准备开始下载 'ds4sd/docling-models' ...
目标保存路径: d:\program\project\python\MHier-RAG\model\docling


d:\program\project\python\MHier-RAG\venv\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\program\project\python\MHier-RAG\venv\lib\site-packages\huggingface_hub\file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 8 files:  25%|██▌       | 2/8 [00:06<00:15,  2.59s/it]'(ReadTimeoutError("HTTPSConnectionPool(host='hf-mirror.com', port=443): Read timed out. (read timeout=10)"), '(Request ID: a15235e1-4d76-4aac-a6e4-45

# Pipeline

In [ ]:
from pipeline import RunConfig, PipelineConfig,Pipeline
preprocess_configs = {"ser_tab": RunConfig(use_serialized_tables=True),
                      "no_ser_tab": RunConfig(use_serialized_tables=False)}

max_st_qwenturbo8k_reasoning_config = RunConfig(
    use_serialized_tables=False,
    parent_document_retrieval=True,
    llm_reranking=True,
    parallel_requests=1,
    submission_name="",
    pipeline_details="",
    api_provider="qwen",
    answering_model="qwen-turbo",
    config_suffix="_max_qwen-turbo1-llmre-reasoning"
)

In [ ]:
from pyprojroot import here
root_path = here() / "data" / "test_set"
print("root_path:", root_path)
pipeline = Pipeline(root_path,
                    run_config=max_st_qwenturbo8k_reasoning_config
                    )

此方法将PDF报告解析为JSON文件。它会在debug/data_01_parsed_reports目录中创建JSON文件。这些JSON文件将在后续步骤中使用
它还会将文档生成的原始输出存储在 debug/data_01_parsed_reports_debug目录中。这些JSON文件包含大量元数据，但不会被使用

In [ ]:
pipeline.parse_pdf_reports_sequential()

此方法将debug/data_01_parsed_reports中的JSON转换为更简单的JSON，即Markdown格式的页面列表
新的JSON文件位于debug/data_02_merged_reports中

In [ ]:
pipeline.merge_reports()

此方法将报告导出为纯 Markdown 格式。这些报告仅用于审阅和全文搜索配置：gemini_thinking_config
新文件位于 debug/data_03_reports_markdown 目录下

In [ ]:
pipeline.export_reports_to_markdown()

此方法将报告分割成多个数据块，用于向量化处理
新的 JSON 文件位于 databases/chunked_reports 目录中。

In [ ]:
pipeline.chunk_reports()

此方法从分块报告中创建向量数据库
新文件位于 databases/vector_dbs 目录中。

In [ ]:
pipeline.create_vector_dbs()

此方法处理问题和答案
问题处理逻辑取决于run_config

In [ ]:
pipeline.process_questions()